In [1]:
import numpy as np
import pandas as pd
import os
# import joblib
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
from trainmodeltiny_createdata import create_training_data_NN_like_micro, get_keras_model_size
from sklearn.metrics import accuracy_score, confusion_matrix

In [2]:
def NeuralNetworkModel(train, test, labelTrain, labelTest):
    train = np.array(train, dtype=np.float32)
    test = np.array(test, dtype=np.float32)
    labelTrain = np.array(labelTrain, dtype=np.int32)
    labelTest = np.array(labelTest, dtype=np.int32)

    if labelTrain.min() == 1:
        labelTrain -= 1
        labelTest -= 1

    num_classes = int(max(labelTrain.max(), labelTest.max()) + 1)

    model = keras.Sequential([
        keras.Input(shape=(train.shape[1],)),
        layers.Dense(8, activation='relu'),
        layers.Dense(8, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])

    opt = keras.optimizers.Adam(learning_rate=0.01)

    model.compile(optimizer=opt,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    model.fit(train, labelTrain, epochs=30, verbose=1, batch_size=16)

    # Đánh giá mô hình trên tập test
    loss, accuracy = model.evaluate(test, labelTest)
    print("Accuracy:", accuracy)

    # Dự đoán trên tập test
    y_pred = np.argmax(model.predict(test), axis=1)

    # Tính toán ma trận nhầm lẫn
    cm = confusion_matrix(labelTest, y_pred)
    print("\nConfusion Matrix:\n", cm)

    # In báo cáo phân loại
    print("\nClassification Report:\n", classification_report(labelTest, y_pred))

    return model


data = pd.read_csv('./input_nano_33/train_div10.csv')
data_test = pd.read_csv('./input_nano_33/test_div10.csv')


total_list_NN, train_labels_NN = create_training_data_NN_like_micro(data=data)
total_list_NN_test, train_labels_NN_test = create_training_data_NN_like_micro(
    data=data_test)

Created 31556 windows × 10 samples (interleaved format)
Created 12749 windows × 10 samples (interleaved format)


In [3]:
nnM = NeuralNetworkModel(train=total_list_NN, test=total_list_NN_test,
                         labelTrain=train_labels_NN, labelTest=train_labels_NN_test)


Epoch 1/30
1973/1973 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9940 - loss: 0.0180
Epoch 2/30
1973/1973 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9981 - loss: 0.0070
Epoch 3/30
1973/1973 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9982 - loss: 0.0063
Epoch 4/30
1973/1973 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.9990 - loss: 0.0032
Epoch 5/30
1973/1973 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9990 - loss: 0.0062
Epoch 6/30
1973/1973 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9994 - loss: 0.0026
Epoch 7/30
1973/1973 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9995 - loss: 0.0022
Epoch 8/30
1973/1973 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9994 - loss: 0.0017
Epoch 9/30
1973/1973 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9989 - loss: 0.0032
Epoch 10/30
1973/1973 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9994 - loss: 0.0028
Epoch 11/30
1973/1973 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.9996 - loss: 0.0032
Epoch 12/30
1973/1973 ━━━━━━━

In [4]:
get_keras_model_size(nnM, "NeuralNetwork")

📦 NeuralNetwork: Model Size = 31.94 KB
🔢 Tổng số tham số huấn luyện: 356
----------------------------------------------------------------------


(31.9375, 356)

In [7]:
# Lấy trọng số của lớp Dense đầu tiên (index 1 trong model.layers)
keras_weights = nnM.layers[1].get_weights() 
# keras_weights[0] là ma trận trọng số, keras_weights[1] là bias
print("--- 1. Kiểm tra mô hình Keras gốc (nnM) ---")
print(f"Kiểu dữ liệu của Ma trận Trọng số Lớp 1 (W): {keras_weights[0].dtype}")
print(f"Kích thước (Shape) của W: {keras_weights[0].shape}")
print("-" * 40)

--- 1. Kiểm tra mô hình Keras gốc (nnM) ---
Kiểu dữ liệu của Ma trận Trọng số Lớp 1 (W): float32
Kích thước (Shape) của W: (8, 8)
----------------------------------------


In [9]:
# joblib.dump(nnM, 'namth.dat')
converter = tf.lite.TFLiteConverter.from_keras_model(nnM)
tflite_model = converter.convert()

# Save the model to disk
open("position.tflite", "wb").write(tflite_model)

basic_model_size = os.path.getsize("position.tflite")
print("Model is %d bytes" % basic_model_size)

INFO:tensorflow:Assets written to: C:\Users\ADMIN\AppData\Local\Temp\tmprz8yam5n\assets


INFO:tensorflow:Assets written to: C:\Users\ADMIN\AppData\Local\Temp\tmprz8yam5n\assets


Saved artifact at 'C:\Users\ADMIN\AppData\Local\Temp\tmprz8yam5n'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 30), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)
Captures:
  2868686367504: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2868686368656: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2868686368272: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2868686369040: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2868686368848: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2868686369424: TensorSpec(shape=(), dtype=tf.resource, name=None)
Model is 3508 bytes


In [14]:
import os
import numpy as np
import tensorflow as tf

def save_tflite(model, converter, out_path):
    tflite_model = converter.convert()
    open(out_path, "wb").write(tflite_model)
    size_b = os.path.getsize(out_path)
    print(f"Saved {out_path}- {size_b} bytes ({size_b/1024:.2f} KB)")
    return out_path, size_b

def representative_data_gen_from_array(arr, num_samples=100):
    """
    arr: numpy array of shape (N, input_dim), dtype float32 (or will be cast)
    Yields samples in the required form for TFLite representative dataset:
        a generator that yields [input_sample.astype(np.float32)]
    """
    # Shuffle to get varied samples
    idx = np.arange(len(arr))
    np.random.shuffle(idx)
    idx = idx[:num_samples]
    for i in idx:
        sample = arr[i]
        # ensure shape (1, input_dim)
        sample = np.asarray(sample, dtype=np.float32).reshape(1, -1)
        yield [sample]

def convert_with_quantizations(keras_model,
                               representative_array=None,
                               out_dir="tflite_models",
                               rep_samples=200):
    os.makedirs(out_dir, exist_ok=True)

    # 1) Baseline (no quantization) - default converter
    converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)
    converter.optimizations = []  # no quant
    print("\n--- Converting baseline (no quantization) ---")
    save_tflite(keras_model, converter, os.path.join(out_dir, "model_baseline.tflite"))

    # 2) Dynamic range quantization (weights quantized to int8, activations float)
    converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    # no representative dataset needed
    print("\n--- Converting dynamic range quantization ---")
    save_tflite(keras_model, converter, os.path.join(out_dir, "model_dynamic_range.tflite"))

    # 3) Float16 quantization (weights to float16) - good for CPUs that support fp16
    converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.target_spec.supported_types = [tf.float16]
    print("\n--- Converting float16 quantization ---")
    save_tflite(keras_model, converter, os.path.join(out_dir, "model_float16.tflite"))

    # 4) Full integer quantization (weights + activations int8)- requires representative dataset
    if representative_array is None:
        print("\n--- Skipping full integer quantization (no representative dataset provided) ---")
    else:
        converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        # supported_ops ensures int8 kernels are used
        converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
        # Set input and output types to int8 (or uint8)
        converter.inference_input_type = tf.int8
        converter.inference_output_type = tf.int8
        # Representative dataset generator
        gen = lambda: representative_data_gen_from_array(representative_array, num_samples=rep_samples)
        converter.representative_dataset = gen

        print("\n--- Converting full integer quantization (int8) ---")
        try:
            save_tflite(keras_model, converter, os.path.join(out_dir, "model_int8.tflite"))
        except Exception as e:
            print("Full integer conversion failed:", e)
            print("Possible reasons: some ops are not quantizable, or representative data shape/dtype mismatch.")

convert_with_quantizations(nnM, representative_array=total_list_NN, out_dir="tflite_models", rep_samples=300)



--- Converting baseline (no quantization) ---
INFO:tensorflow:Assets written to: C:\Users\ADMIN\AppData\Local\Temp\tmp_2kgv9zh\assets


INFO:tensorflow:Assets written to: C:\Users\ADMIN\AppData\Local\Temp\tmp_2kgv9zh\assets


Saved artifact at 'C:\Users\ADMIN\AppData\Local\Temp\tmp_2kgv9zh'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 30), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)
Captures:
  2868686367504: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2868686368656: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2868686368272: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2868686369040: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2868686368848: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2868686369424: TensorSpec(shape=(), dtype=tf.resource, name=None)
Saved tflite_models\model_baseline.tflite- 3508 bytes (3.43 KB)

--- Converting dynamic range quantization ---
INFO:tensorflow:Assets written to: C:\Users\ADMIN\AppData\Local\Temp\tmpn0kffr5e\assets


INFO:tensorflow:Assets written to: C:\Users\ADMIN\AppData\Local\Temp\tmpn0kffr5e\assets


Saved artifact at 'C:\Users\ADMIN\AppData\Local\Temp\tmpn0kffr5e'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 30), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)
Captures:
  2868686367504: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2868686368656: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2868686368272: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2868686369040: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2868686368848: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2868686369424: TensorSpec(shape=(), dtype=tf.resource, name=None)
Saved tflite_models\model_dynamic_range.tflite- 3508 bytes (3.43 KB)

--- Converting float16 quantization ---
INFO:tensorflow:Assets written to: C:\Users\ADMIN\AppData\Local\Temp\tmp6c4acryy\assets


...

Saved artifact at 'C:\Users\ADMIN\AppData\Local\Temp\tmp6c4acryy'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 30), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)
Captures:
  2868686367504: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2868686368656: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2868686368272: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2868686369040: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2868686368848: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2868686369424: TensorSpec(shape=(), dtype=tf.resource, name=None)
Saved tflite_models\model_float16.tflite- 3572 bytes (3.49 KB)

--- Converting full integer quantization (int8) ---
INFO:tensorflow:Assets written to: C:\Users\ADMIN\AppData\Local\Temp\tmp52h2qcdl\assets


INFO:tensorflow:Assets written to: C:\Users\ADMIN\AppData\Local\Temp\tmp52h2qcdl\assets


Saved artifact at 'C:\Users\ADMIN\AppData\Local\Temp\tmp52h2qcdl'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 30), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)
Captures:
  2868686367504: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2868686368656: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2868686368272: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2868686369040: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2868686368848: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2868686369424: TensorSpec(shape=(), dtype=tf.resource, name=None)


C:\Users\ADMIN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\tensorflow\lite\python\convert.py:863: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


Saved tflite_models\model_int8.tflite- 3272 bytes (3.20 KB)
